# Part 2: Image processing & Transfer learning
This notebook contains the top performing CNN model and the autoencoding model evaluated on Block 2 data from the CIFAR-100 dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

In [ ]:
from keras.datasets import cifar100
import os

# load CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

# randomly select 50-50 classes for Block 1 & Block 2
all_classes = np.unique(y_train)
np.random.shuffle(all_classes)
block1_classes = all_classes[:50]
block2_classes = all_classes[50:100]

# filter data for Block 2
block2_train_indices = np.isin(y_train, block2_classes).flatten()
x_block2_train = x_train[block2_train_indices]
y_block2_train = y_train[block2_train_indices]

# split data into train-validation for Block 2
split_index = int(len(x_block2_train) * 0.8)
x_block2_val = x_block2_train[split_index:]
y_block2_val = y_block2_train[split_index:]
x_block2_train = x_block2_train[:split_index]
y_block2_train = y_block2_train[:split_index]

# encode labels to range from 0 to 49 for both blocks
y_block2_train_encoded = (y_block2_train - block2_classes.min()) % 50
y_block2_val_encoded = (y_block2_val - block2_classes.min()) % 50

# normalize pixel values
x_train_block2 = x_block2_train.astype('float32') / 255
x_val_block2 = x_block2_val.astype('float32') / 255


## Model 1 - CNN model with residual block

In [ ]:
# load model
cnn_model = load_model("models/skip_connect.keras")

# evaluate model
cnn_loss, cnn_accuracy = cnn_model.evaluate(x_val_block2, y_block2_val_encoded)

print("Test Loss:", cnn_loss)
print("Test Accuracy:", cnn_accuracy)

## Model 2 - Autoencoder model

In [ ]:
# load model
auto_model = load_model("models/autoencode.keras")

# evaluate model
evaluation_loss = auto_model.evaluate(x_val_block2, x_val_block2, verbose=2)
print("Evaluation Loss:", evaluation_loss)

# reconstruct images
def vis_reconstruct_img(model, original_img, n=3):
    reconstructed_images = model.predict(original_img)
    n = 3 
    plt.figure(figsize=(9, 4))
    for i in range(n):
        # og images
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original_img[i])
        plt.title("Original")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # reconstructed images
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed_images[i])
        plt.title("Reconstructed")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

vis_reconstruct_img(auto_model, x_val_block2)